# Task4 基于深度学习的文本分类1
----
在上一章节，我们使用传统机器学习算法来解决了文本分类问题，从本章开始我们将尝试使用深度学习方法。

## 基于深度学习的文本分类
----
与传统机器学习不同，深度学习既提供特征提取功能，也可以完成分类的功能。从本章开始我们将学习如何使用深度学习来完成文本表示。

### 学习目标
- 学习FastText的使用和基础原理
- 学会使用验证集进行调参

###文本表示方法 Part2
**现有文本表示方法的缺陷**
在上一章节，我们介绍几种文本表示方法：

- One-hot
- Bag of Words
- N-gram
- TF-IDF

也通过sklean进行了相应的实践，相信你也有了初步的认知。但上述方法都或多或少存在一定的问题：转换得到的向量维度很高，需要较长的训练实践；没有考虑单词与单词之间的关系，只是进行了统计。

与这些表示方法不同，深度学习也可以用于文本表示，还可以将其映射到一个低纬空间。其中比较典型的例子有：FastText、Word2Vec和Bert。在本章我们将介绍FastText，将在后面的内容介绍Word2Vec和Bert。

### FastText
FastText是一种典型的深度学习词向量的表示方法，它非常简单通过Embedding层将单词映射到稠密空间，然后将句子中所有的单词在Embedding空间中进行平均，进而完成分类操作。

所以FastText是一个三层的神经网络，输入层、隐含层和输出层。

![](https://camo.githubusercontent.com/4e01004146c81db5ee15df1b373374b3ff145bfa/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303731343230343835363538392e706e67)

下图是使用keras实现的FastText网络结构：

In [4]:
!pip install keras
!pip install tensorflow

  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=45919 sha256=43a96af97ee3bdacef64b6b6bf95e0e21a783ae74451484bdb9b0485c656d87f
  Stored in directory: /root/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc
Successfully built pyyaml
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
# coding: utf-8
from __future__ import unicode_literals

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dense


VOCAB_SIZE = 2000
EMBEDDING_DIM = 100
MAX_WORDS = 500
CLASS_NUM = 5

def build_fastText():
    model = Sequential()
    # 通过embedding层，我们将词汇映射成EMBEDDING_DIM维向量
    model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_WORDS))
    # 通过GlobalAveragePooling1D，我们平均了文档中所有词的embedding
    model.add(GlobalAveragePooling1D())
    # 通过输出层Softmax分类（真实的fastText这里是分层Softmax），得到类别概率分布
    model.add(Dense(CLASS_NUM, activation='softmax'))
    # 定义损失函数，优化器，分类度量指标
    model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

if __name__=='__main__':
    model = build_fastText()
    print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 100)          200000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 505       
Total params: 200,505
Trainable params: 200,505
Non-trainable params: 0
_________________________________________________________________
None


FastText在文本分类任务上，是优于TF-IDF的：

- FastText用单词的Embedding叠加获得的文档向量，将相似的句子分为一类
- FastText学习到的Embedding空间维度比较低，可以快速进行训练

如果想深度学习，可以参考论文：

Bag of Tricks for Efficient Text Classification, https://arxiv.org/abs/1607.01759

基于FastText的文本分类
FastText可以快速的在CPU上进行训练，最好的实践方法就是官方开源的版本： https://github.com/facebookresearch/fastText/tree/master/python

- pip安装

In [2]:
!pip install fasttext

     |################################| 68 kB 7.3 MB/s  eta 0:00:01
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3024732 sha256=2485d623748288ceea00c042483d4b365c2157c8c2b4ddf07deb6408708e9283
  Stored in directory: /root/.cache/pip/wheels/c3/5c/d0/4a725c6ee7df3267d818d3bc9d89bb173b94832f2b9eca6368
Successfully built fasttext
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


- 源码安装

两种安装方法都可以安装，如果你是初学者可以优先考虑使用pip安装。

- 分类模型

In [12]:
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')

import fasttext
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=25, loss="hs")

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))
# 0.82

0.8252595163033803


此时数据量比较小得分为0.82，当不断增加训练集数量时，FastText的精度也会不断增加5w条训练样本时，验证集得分可以到0.89-0.90左右。

### 如何使用验证集调参
在使用TF-IDF和FastText中，有一些模型的参数需要选择，这些参数会在一定程度上影响模型的精度，那么如何选择这些参数呢？

- 通过阅读文档，要弄清楚这些参数的大致含义，那些参数会增加模型的复杂度
- 通过在验证集上进行验证模型精度，找到模型在是否过拟合还是欠拟合

![](https://camo.githubusercontent.com/3c19cda9d91954875be0b59abe99fad024552d29/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303731343230343430333834342e706e67)

这里我们使用10折交叉验证，每折使用9/10的数据进行训练，剩余1/10作为验证集检验模型的效果。这里需要注意每折的划分必须保证标签的分布与整个数据集的分布一致。

通过10折划分，我们一共得到了10份分布一致的数据，索引分别为0到9，每次通过将一份数据作为验证集，剩余数据作为训练集，获得了所有数据的10种分割。不失一般性，我们选择最后一份完成剩余的实验，即索引为9的一份做为验证集，索引为1-8的作为训练集，然后基于验证集的结果调整超参数，使得模型性能更优。

### 本章小结
本章介绍了FastText的原理和基础使用，并进行相应的实践。然后介绍了通过10折交叉验证划分数据集。

### 本章作业
阅读FastText的文档，尝试修改参数，得到更好的分数
基于验证集的结果调整超参数，使得模型性能更优

#### 训练数据和验证数据加载

In [17]:
import fasttext
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('../input/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')

#### 调参训练模型

In [9]:
# model = fasttext.train_supervised('train.csv', lr=0.8, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=30, loss="ns")
# val_pred = [model.predict(x,k=3)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8758633612084037

# model = fasttext.train_supervised('train.csv', lr=0.9, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=30, loss="ns")
# val_pred = [model.predict(x,k=3)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8764273074705968
# model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=30, loss="hs")
# val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8242632865948475
# model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=30, loss="ova")
# val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8790382771320723

# model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=30, loss="ns")
# val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.879371994348176

# model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=30, loss="softmax")
# val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8800869271492763

# model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
#                                   verbose=2, minCount=1, epoch=38, loss="softmax")
# val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8810064789241421(0.8784835670454213 不太稳定)
# model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
#                                   verbose=3, minCount=1, epoch=38, loss="softmax")
# val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
# 0.8815890757291062
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=3, minCount=1, epoch=38, loss="softmax")

#### 比较列表(fasttext)
|参数（lr）|参数（wordNgrams）|参数（verbose）|参数（epoch）|参数（loss）|得分|
|:----|:----|:----|:----|:----|:----|
|0.8|2|2|30|ns|0.8758633612084037|
|0.9|2|2|30|ns|0.8764273074705968|
|1.0|2|2|30|hs|0.8242632865948475|
|1.0|2|2|30|ova|0.8790382771320723|
|1.0|2|2|30|ns|0.879371994348176|
|1.0|2|2|30|softmax|0.8800869271492763|
|1.0|2|2|38|softmax|0.8810064789241421|
|1.0|2|3|38|softmax|0.8815890757291062|

In [10]:
model.save_model("model_fastText.bin")

加载训练好的模型文件，进行预测

In [19]:
model = fasttext.load_model("model_fastText.bin")
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]

In [20]:
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))

0.8829122226599748


### 参赛格式如下：

#### 数据读取

In [1]:
import fasttext
import pandas as pd

# 读取训练集数据
train_df = pd.read_csv('../input/train_set.csv', sep='\t')
# 滤除缺失数据
train_df = train_df.dropna()

# 读取测试集数据
test_df = pd.read_csv('../input/test_a.csv', sep='\t')
# 滤除缺失数据
test_df = test_df.dropna()

# 整理为fastText要求的文本格式，并生成训练数据
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:].to_csv('train.csv', index=None, header=None, sep='\t')

### 训练模型并保存

In [ ]:
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=3, minCount=1, epoch=45, loss="softmax")

model.save_model("model_fastText.bin")

### 输出预测结果

In [3]:
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df.iloc[:]['text']]

df = pd.DataFrame()
df['label'] = val_pred
df.to_csv('submit.csv',index=None)

### 我提交的得分：0.9309